# flowField class for iterative solution

## Contents
- Subclass of numpy.ndarray()

### Basic flow descriptors
Define as a dictionary:
* $ \alpha, \beta $  describing the periodicity, and $Re$
* K,L,M,N describing the resolution:
    - K: No. of temporal frequencies
    - L: No. of positive streamwise Fourier modes
    - M: No. of positive spanwise Fourier modes
    - N: No. of Chebyshev collocation nodes in wall-normal 
    - The total size of each variable would be (2L+1)(2M+1)N
* Flow type: Couette or Poiseuille (use flags)- default to Couette
    - Boundary conditions on $u_0$
    - dPdx
    
### Values
* Values (This shouldn't be directly accessible- use reshapers to access values)
    - Store only fluctuations
    - Define a separate $u_{base}$


### Methods to be defined
Should I write these as a view() or simply as a reshape()?
* Viewers: 
    - 1D array for state
    - 4D array (streamwise, spanwise, variable ID, wall-normal)
* Truncating:
    - Returning only (u,w)
    - Returning only interior nodes, ignoring the walls
* Dot product
* Norm
* spectral2physical:
    - Do not change the flowField class itself, instead, write a new object
    
## Inputs
* Flow descriptors (see above)
    - Keep $ \alpha, \beta, Re$ as optional parameters, default values being those used in ChannelFlow: (1.14,2.5,400)
    * Resolution: L,M,N    
* Base flow type (flag):
    - Either linear, or quadratic. Assume linear as default. 
* Read from file- optional (flag):
    - Filename, format ('mat', 'npy', 'asc',...)
* Noise levels (default to zero):
    - Add random noise to the state vector whose norm is that as input.

## Tests for class
* Ensure that solving for flat-walled Couette and Poiseuille flows using direct inversion works
* Ensure that the above cases work with iterative solver. 

### Other notes to self:
* Write doc-strings for all the methods and inputs
* When initializing flowField as white-noise, think about smoothening the noise. 


## Collaborator comments
Use this cell for comments

---------------------------------------------------

## Inheriting numpy.ndarray()

Reference: http://docs.scipy.org/doc/numpy/user/basics.subclassing.html

In [1]:
import numpy as np

class C(np.ndarray): pass

arr = np.zeros(3,)
c_arr = arr.view(C)

print(type(arr))
print(type(c_arr))
print(c_arr)



<class 'numpy.ndarray'>
<class '__main__.C'>
[ 0.  0.  0.]


In [2]:
v= c_arr[:]
print(type(v))

v is c_arr


<class '__main__.C'>


False

In [3]:
print(type(v))

<class '__main__.C'>


In [5]:
print(dir())

['C', 'In', 'Out', '_', '_2', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_ih', '_ii', '_iii', '_oh', '_sh', 'arr', 'c_arr', 'exit', 'get_ipython', 'np', 'quit', 'v']


In [6]:
print(c_arr)
v[1]=5.
print(c_arr)

[ 0.  0.  0.]
[ 0.  5.  0.]


In [7]:
v == c_arr

C([ True,  True,  True], dtype=bool)

Need to be careful when using `is`. It refers to the IDs of objects. If using arrays or elements of arrays, using `is` is a bad idea. Stick to `==` when comparing numerical values- which is usually all I care for.

In [8]:
gen = np.zeros(3,)
v1 = gen[:]
print(v1 is gen)
v1[1]=3.
print(gen)
print(gen[:] is gen)
print(id(gen))
print(id(gen[:]))
v2 = gen
print (v2 is gen)
print (id(v2))
print(v1 == gen)
print (v1[0] is gen[0])

False
[ 0.  3.  0.]
False
34815088
34837296
True
34815088
[ True  True  True]
False


## Reading files into dictionary


In [9]:
ls


flowConfig.txt  flowField.ipynb


In [10]:
%cat flowConfig.txt

alpha   1.14	% (float) Streamwise wavenumber of periodic box
beta    2.5	% (float) Spanwise wavenumber of periodic box
omega   0.0	% (float) Fundamental frequency resolved in computation
ReLam   400.0 	% (float) Re based on laminar base flow
isPois  0	% (0 or 1) 1 for Poiseuille flow, 0 for Couette
noise   0.0	% (float) Norm of white noise to be added
L       23      % (int) Number of positive streamwise Fourier modes 
M       23	% (int) Number of positive spanwise Fourier modes
N       35	% (int) Number of Chebyshev collocation nodes
K       0	% (int) Number of positive Fourier frequency modes
--- 0 ---- ---- WARNING: No text goes after this line. DO NOT EDIT THIS LINE AT ALL.


In [11]:
flowDict = {}
with open("flowConfig.txt",'r') as f:
    for line in f:
        (key,val) = line.split()[:2]
        flowDict[key] = float(val)


In [12]:
defaultDict = {'alpha':1.14, 'beta' : 2.5, 'omega':0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'K':0.0,
               'ReLam': 400.0, 'isPois':0.0, 'noise':0.0 , 'testvar':5.5}
for key in defaultDict:
    if key not in flowDict:
        flowDict[key] = defaultDict[key]
        print(key,flowDict[key],type(flowDict[key]))

testvar 5.5 <class 'float'>


In [13]:
print(flowDict)
print( 2.5+flowDict['alpha'])

{'omega': 0.0, 'noise': 0.0, 'K': 0.0, 'ReLam': 400.0, 'isPois': 0.0, 'testvar': 5.5, 'beta': 2.5, 'alpha': 1.14, 'M': 23.0, 'N': 35.0, '---': 0.0, 'L': 23.0}
3.6399999999999997


In [14]:
flowDict['alpha'],flowDict['beta']
print (2+float(flowDict['alpha']))
print(flowDict.values)
float(flowDict['beta'])

flowDict['alpha'] = float(flowDict['alpha'])
flowDict['alpha']+2.5

3.1399999999999997
<built-in method values of dict object at 0x7f11a8fde4c8>


3.6399999999999997

## Defining flowField class that inherits np.ndarray

For starters, defining the class to initialize an empty ndarray along with a dictionary provided during initialization.

I need to verify that the dictionary being supplied has all the info I need to go with a flowField class. But, I can't verify this all the time. This is how I'll deal with it: 
* Verify the dictionary only when constructing a flowField class. So, that's when
    - Explicitly constructing an instance of the class
    - Viewing a given array as an instance of the class
* When slicing an array of the flowField class (to obtain another instance of the class), don't bother with the check. 

Basically, ensure every instance of the flowField class has a valid dictionary, and then stop worrying about it. 

So, we need a function that verifies the validity of dictionary, or creates one if one isn't supplied. The parameters required in the dictionary, and their defaults, are as follows:
* alpha : 1.14    
    *Wavenumber of fundamental streamwise Fourier mode*
* beta :  2.5    
    *Wavenumber of fundamental spanwise Fourier mode*
* omega: 0.0    
    *Fundamental frequency*
* ReLam: 400.0  
    *Reynolds number of the laminar base flow*
* isPois: 0     
    *Flag for base flow type. 0: Couette, 1: Poiseuille*
* noise: 0.0    
    *Norm of noise to be added to the flow*
* L: 0          
    *Number of (positive) harmonics of fundamental streamwise Fourier mode.*
* M: 0          
    *Number of (positive) harmonics of fundamental spanwise Fourier mode*
* N: 35         
    *Number of Chebyshev collocation nodes*
* K: 0         
    *Number of (positive) harmonics of fundamental frequency*

** For now, I'm ignoring the fact that np.ndarray can be viewed as a subclass. I'm defining the dictionary and checks in flowField.__new__(). Later, I'll have to move this to __array_finalize__ so that the dictionary is defined for cases when either an explicit constructor call is made or a view-casting is done**

### Ensure these features:
* View-casting:
    - Take an existing numpy array and view it as a flowField class, given that a flowDict dictionary is supplied.
* Explicit construction:
    - Construct a randomField object based on noise levels supplied
* New from template:
    - Truncate and expand an instance of flowField to obtain a new instance with a changed dictionary (to reflect the truncation of L,M,N, or K)



** I have three cases to consider when building a flowField instance. Suppose L = 1**:
- The flowField has streamwise wavenumbers $-\alpha, 0, \alpha$
- The flowField only has streamwise wavenumbers $-\alpha, \alpha$
- The flowField only has streamwise wavenumber $\alpha$
    
This question becomes particularly important if only a half-plane or half-volume of the wavenumber space is considered. 

This is how I chose to resolve this issue. Considering L:
* If L = 0, then the flowField is of wavenumber $\alpha$
* If L = n ($\in \mathbb{N}$), then the flowField is resolved in wavenumbers {$0,\alpha,2\alpha,..,n\alpha$}.
* If L = -n ($n \in \mathbb{N}$), then the flowField is resolved in wavenumbers {$-n\alpha, (-n+1)\alpha,..,0,\alpha,...,n\alpha$}

This way, a flowField can be defined in three different ways for each of the three Fourier axes (streamwise, spanwise, temporal):
* As just one Fourier mode, $\alpha$ (which could be positive, negative, or zero). 
* As a collection of `n` harmonics (positive integer multiples of a positive/negative wavenumber), along with mode zero (the invariant mode).
* As a collection of `2n+1` harmonics, from $-n\alpha$ through $n\alpha$.

** Correction: Cannot do flowFields with just modes 

I think this should be enough to deal with most cases. We will later need to define collections of flowFields. 

In [197]:
# In case dictionary is 'None':
defaultDict = {'alpha':1.14, 'beta' : 2.5, 'omega':0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'K':0.0,
               'ReLam': 400.0, 'isPois':0.0, 'noise':0.0 }

def verify_dict(tempDict):
    if tempDict is None:
        tempDict = defaultDict
    else: 
        for key in defaultDict:
            if key not in tempDict:
                tempDict[key] = defaultDict[key]
    return tempDict

flowDict = None
flowDict = verify_dict(flowDict)
print(flowDict)

{'K': 0.0, 'N': 0.0, 'M': 0.0, 'omega': 0.0, 'noise': 0.0, 'L': 0.0, 'alpha': 1.14, 'beta': 2.5, 'ReLam': 400.0, 'isPois': 0.0}


## Slice
I'm defining `slice` as a general method used for either truncating or extending the existing flowField instance. 

There are several cases to consider:
1.  Both existing and requested instances have the same wavenumber extension: either both contain positive and negative wavenumbers, or both contain only non-negative wavenumbers. For this case, there are 2 subcases:
    1.  The requested instance needs the existing instance to be truncated or held the same.
    2.  The requested instance needs the existing instance to be extended.
2.  The existing instance contains positive as well as negative wavenumbers, whereas the requested instance is only for positive wavenumbers. For this case:
    1.  The requested instance requires truncation in positive modes of the existing instance.
    2.  The requested instance requires extension in positive wavenumbers (with zeros for modes unavailable).
3.  The existing instance contains only positive wavenumbers, whereas the requested instance is for positive as well as negative. For this case too:
    1.  Truncation.
    2.  Extension.


## Going from half plane/volume to full using conjugate
I thought that if the Fourier modes available were $(0:K\omega,0:L\alpha, 0:M\beta)$, I could extend each set of modes into the negative side. But that's not true, I can only do that for just 1 set of modes. That is, I can't extend a quadrant to a full plane or an octant to a full volume. I can only go from 2 quadrants to 4, or 4 octants to 8. 

For now, I will suppose that we always have the flow resolved in the half-plane or the half-volume $\beta>0$. So, K,L are always negative (-0=0). Only M is allowed to be positive, but it could also be set to be negative. 

**With this simplification, cases 2 and 3 identified for slicing (above) only relate to M. For K and L, it's always case 1.**



In [1]:
import numpy as np
import scipy as sp
from scipy.linalg import norm
from warnings import warn
#from pseudo.py import chebint

defaultDict = {'alpha':1.14, 'beta' : 2.5, 'omega':0.0, 'L': 23.0, 'M': 23.0, 'N': 35.0, 'K':0.0,
               'ReLam': 400.0, 'isPois':0.0, 'noise':0.0 }

def verify_dict(tempDict):
    '''Verify that the supplied flowDict has all required parameters'''
    if tempDict is None:
        tempDict = defaultDict
    else: 
        for key in defaultDict:
            if key not in tempDict:
                tempDict[key] = defaultDict[key]
    return tempDict

def read_dictFile(dictFile):
    '''Read flowDict from file. MUST use "flowConfig.txt" as template. '''
    tempDict = {}
    with open("flowConfig.txt",'r') as f:
        for line in f:
            (key,val) = line.split()[:2]
            tempDict[key] = float(val)    
    return tempDict

class flowField(np.ndarray):
    ''' Provides a class to define u,v,w,p in 4D: time, x,z,y. 
    Ordered as (omega,alpha,beta,nd,y): omega, alpha, beta are Fourier modes in t,x,z respectively.
    nd is an index going from 0 to 3 for u,v,w,p. 
    y is the array of Chebyshev collocation nodes
    Methods: 
        slice(K,L,M,nd,N): Make gride finer or coarser along any direction
        extract(K,L,M,nd,N): Similar to slice, but allows only for choosing existing Fourier modes (slice also allows for extrapolation)'''
    def __new__(cls, arr=None, flowDict=None, dictFile= None):
        # The shape of the numpy array comes from the dictionary
        #   so it is absolutely necessary to have the dictionary
        print('constructor has been called')
        if flowDict is None:
            if dictFile is not None:
                flowDict = verify_dict(read_dictFile(dictFile))
            else:
                flowDict = defaultDict
        else:
            flowDict = verify_dict(flowDict)
        
        L = int(flowDict['L'])
        M = int(flowDict['M'])
        N = int(flowDict['N'])
        K = int(flowDict['K'])
        nt = int(3.*abs(K)/2. - K/2. + 1)    # = 1 if K=0;    = K+1 if K>0;    = 2*|K|+1 if K<0
        nx = int(3.*abs(L)/2. - L/2. + 1)    
        nz = int(3.*abs(M)/2. - M/2. + 1) 
        
        
        #obj = np.ndarray.__new__(cls, shape=(nt,nx,nz,4,N),dtype=float) # The 4th index goes to 4 because u,v,w,p
        if arr is None:
            obj = np.zeros((nt,nx,nz,4,N),dtype=np.complex).view(cls)
        else:
            obj = np.asarray(arr).view(cls)

        return obj
    
    def __array_finalize__(self,obj,dictFile=None,flowDict=defaultDict):
        if self.dtype != np.complex:
            warn('flowField class is designed to work with complex array entries\n'+
                 'To obtain real/imaginary parts of an instance, use class methods "real" and "imag"')
        if type(obj) is type(self):
            if obj.shape != self.shape:
                warn('Extracting slice of np.ndarray does not effect changes in flowDict.\n'+ 
                        'Use flowField method "slice" instead')
            return
        if (obj is None) or (type(obj) is not np.ndarray):
            raise RuntimeError('Use explicit constructor call when creating class instances without supplying np.ndarrays')
            return
        
        if dictFile is not None:
            flowDict = read_dictFile(dictFile)
            flowDict = verify_dict(flowDict)
        
        L = int(flowDict['L'])
        M = int(flowDict['M'])
        N = int(flowDict['N'])
        K = int(flowDict['K'])
        
        nx = 2*L+1     
        nz = int(3.*abs(M)/2. - M/2. + 1)    # = 1 if K=0;    = K+1 if K>0;    = 2*|K|+1 if K<0
        nt = 2*K+1
        
        if obj.size != (nx*nz*nt*4*N):
            raise RuntimeError('The parameters in the dictionary are not consistent with the size of the supplied array')
        
        self.flowDict = flowDict
        self.nx = nx
        self.nz = nz
        self.nt = nt
        self.N = N
        self.nd = 4
        return

    


In [ ]:
    def slice(self,K=None,L=None,M=None,nd=None,N=None):
        '''
        Returns a class instance with increased/reduced K,L,M,nd,N
        Call as new_inst = myFlowField.slice((Knew,Lnew,None,None,Nnew)) to change values of K,L,N without affecting M and nd
        When the number of Fourier modes (K,L,M, or nt,nx,nz) are smaller than what is requested, 
            additional zero modes are added. For Chebyshev nodes, interpolation is used'''
        if K:
            Kt = self.flowDict['K']               # Temporary name for 'L' of self
            nx = int(3.*abs(L)/2. - L/2. + 1)     # = 1 if L=0;    = L+1 if L>0;    = 2*|L|+1 if L<0
            
            if abs(L) <= abs(Lt): # Case 1.A
                nx0 = int((abs(Lt)-Lt)/2)
                nxm1 = nx0 - int((abs(L)-L)/2)
                nxp1 = nx0 + abs(L) + 1
                obj = obj[:,nxm1:nxp1]
            else:  # Case 1.B
                nxplus = int(abs(L)-abs(Lt))
                if L>0: 
                    obj = np.append(obj, np.zeros((self.nt,nxplus,self.nz,self.nd,self.N),dtype=np.complex),axis=1)
                    obj = np.append(obj[:,:0:-1].conjugate())
            
        if L:
            Lt = self.flowDict['L']               # Temporary name for 'L' of self
            nx = int(3.*abs(L)/2. - L/2. + 1)     # = 1 if L=0;    = L+1 if L>0;    = 2*|L|+1 if L<0
            
            if abs(L) <= abs(Lt): # Case 1.A
                nx0 = int((abs(Lt)-Lt)/2)
                nxm1 = nx0 - int((abs(L)-L)/2)
                nxp1 = nx0 + abs(L) + 1
                obj = obj[:,nxm1:nxp1]
            else:  # Case 1.B
                nxplus = int(abs(L)-abs(Lt))
                if L>0: 
                    obj = np.append(obj, np.zeros((self.nt,nxplus,self.nz,self.nd,self.N),dtype=np.complex),axis=1)
                    obj = np.append(obj[:,:0:-1].conjugate())
            elif L > 0:          # Case 2
                if abs(L) <= abs(Lt): # Case 2.A
                    nx0 = int((abs(Lt)-Lt)/2)
                    nxp1 = nx0 + L + 1
                    obj = obj[:,nx0:nxp1]
                else:  # Case 2.B
                    pass   # Need to add zero modes here
            else: 
                if abs(L) <= abs(Lt):
        return 
            

In [2]:
ff = flowField()
print(ff.dtype)

constructor has been called
complex128


In [3]:
ff1=np.real(ff)
print(ff1.shape)
print(ff1.dtype)


(1, 47, 24, 4, 35)
float64


/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:69: UserWarning: flowField class is designed to work with complex array entries
To obtain real/imaginary parts of an instance, use class methods "real" and "imag"


In [ ]:
cat flowConfig.txt

In [195]:
def tfun(newShape=(None,None,None)):
    for k in newShape[:2]:
        if k:
            print('well')
        else:
            print('nope')
    if newShape[2] > 2:
        warn('Not allowed')
            

In [196]:
tfun((1,None,3))

well
nope


/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:8: UserWarning: Not allowed
